In [ ]:
import os
import uuid
import hashlib
from urllib.parse import urlencode
import json
import requests
from shutil import copyfile
import pytz
import datetime
import time
from threading import Thread

#업비트
import jwt
import upbit_key

#텔레그램
import telegram
from telegram.ext import Updater, CommandHandler 
import telegram_token

class HMTB : # 호랑 말코 타임 봇
    
    WS = {"working" : "입출금 가능", "withdraw_only" : "출금만 가능", "deposit_only" : "입금만 가능", "paused" : "입출금 중단", "unsupported" : "입출금 미지원"}
    
    # 초기 세팅 : init, mktkn, currency_info_update
    def __init__(self) :
        #나중에 파일로 바꾸기
        self.__INFO = {"AK" : upbit_key.ak, "SK" : upbit_key.sk, "URL" : "https://api.upbit.com"}
        self.__AC = "input_AC"
        self.__SK = "input_SK"
        self.__payload = {
            'access_key': self.__INFO['AK'],
            'nonce': str(uuid.uuid4()),
        }
        self.__mktkn__() # 토큰 생성
        self.__currency_info_update()
        self.__chat_updater.dispatcher.add_handler(CommandHandler('start', self.__t_start)) # 핸들러 등록
        self.__chat_updater.dispatcher.add_handler(CommandHandler('all', self.__t_all_info)) # 핸들러 등록

    # 토큰 생성
    def __mktkn__(self) : 
        self.__jwt_token = jwt.encode(self.__payload, self.__INFO["SK"]) # bearer 인증 방식
        self.__authorize_token = 'Bearer {}'.format(self.__jwt_token)
        self.__headers = {"Authorization": self.__authorize_token} 
        self.__token = telegram_token.TELEGRAM_TOKEN # 텔레그램 토큰 값
        self.__bot = telegram.Bot(token=self.__token) # 텔레그램 봇 인스턴스
        self.__chat_updater = Updater(token = self.__token) # 텔레그램 업데이터 인스턴스
        
    # 암호 화폐 정보
    def __currency_info_update(self) : 
        ciu_res = requests.get(self.__INFO["URL"] + "/v1/status/wallet", headers=self.__headers)
        ciu_res_json = ciu_res.json()
        ciu_ret = { i['currency'] : i['wallet_state'] for i in ciu_res_json }
        if not os.path.isfile("currency_new.json") : 
            #print("처음에만 실행됨 / 파일 없음 확인.")
            with open("currency_new.json", "w") as jf:
                json.dump(ciu_ret, jf)
        else :                                  
            copyfile("currency_new.json", "currency_old.json")
            with open("currency_new.json", "w") as jf:
                json.dump(ciu_ret, jf)
    
    # 텔레그램 핸들러 : __t_start, __t_all_info
    def __t_start(self, update, context) :
        update.message.reply_text("Alpha v1.0\ntype ''/help'")
        #print(update.message.chat_id)
        if not os.path.isfile("user_chat_id.txt") : 
            #print("처음에만 실행됨2 / 파일 없음 확인.")
            with open("user_chat_id.txt", "w") as f:
                f.write(f'{update.message.chat_id}\n')
        else :
            with open("user_chat_id.txt", "r+") as f:
                user_ids = f.readlines()
                if str(update.message.chat_id)+'\n' not in user_ids : 
                    f.write(f'{update.message.chat_id}\n')
        
    def __t_all_info(self, update, context) :
        with open("currency_new.json", "r") as jf:
            data = json.load(jf)
            res = ""
            for i, j in data.items() :
                res += f'{i} {self.WS[j]}\n'
        update.message.reply_text(res)
    
    # 코인 입출금 상태 비교 함수. 1분마다 실행. 쓰레드용
    def thread_run(self) : 
        while True : 
            time.sleep(10)
            #print("test")
            self.__currency_info_update()
            res = ""
            with open('currency_new.json', 'r') as fn, open('currency_old.json', 'r') as fo:
                new_data = json.load(fn)
                old_data = json.load(fo)
                new_data_items = list(new_data.items())
                #새 화폐 정보 기준으로 먼저 검색
                for currency, state in new_data_items : 
                    if old_data.get(currency) == None : #새로 생긴 화폐
                        res += f'코인 : {currency}\n새로 생성 : {self.WS[state]}\n'
                        new_data.pop(currency)
                        continue
                    elif new_data[currency] != old_data[currency] :
                        res += f'코인 : {currency}\n상태 변경됨 : {self.WS[state]}\n'
                    new_data.pop(currency)
                    old_data.pop(currency)
                #이전 화폐 기준으로 누락된거 검색 / new_data에 남는거 없어야함. 확인용으로 다 지웠음
                for currency in old_data :
                    res += f'코인 : {currency}\n없어짐\n'

            #print(res)
            if res : 
                today = datetime.datetime.now(pytz.timezone('UTC'))
                res += f'{today.year}-{today.month}-{today.day} {today.hour}:{today.minute} GMT'
                with open("user_chat_id.txt", "r") as f:
                    users = f.readlines()
                    for user in users : 
                        self.__bot.sendMessage(chat_id=user[:-1], text=res)

    # 프로그램 동작
    def HMTB_run(self) : 
        # 코인 상태 변화 감지하는 쓰레드
        th1 = Thread(target=self.thread_run)
        th1.start()
        # Starts polling updates from Telegram.
        self.__chat_updater.start_polling()
        self.__chat_updater.idle()
        
        
def main() :
    h = HMTB()
    h.HMTB_run()

if __name__ == '__main__' : 
    main()